In [1]:
import os
from argparse import ArgumentParser
from sklearn.externals import joblib
from tictacs import from_recipe
from pan import ProfilingDataset
import dill
import cPickle as pickle
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix


infolder = "../DATA/pan16-author-profiling-training-dataset-2016-04-25/pan16-author-profiling-training-dataset-english-2016-02-29/"
outfolder = "models/"
print('Loading dataset->Grouping User texts.\n')
dataset = ProfilingDataset(infolder)
print('Loaded {} users...\n'.format(len(dataset.entries)))
# get config
config = dataset.config
tasks = config.tasks
print('\n--------------- Thy time of Running ---------------')
for task in tasks:
    print('Learning to judge %s..' % task)
    # load data
    X, y = dataset.get_data(task)

Loading dataset->Grouping User texts.

Loaded 436 users...


--------------- Thy time of Running ---------------
Learning to judge age..
Learning to judge gender..


In [ ]:
Instance

In [ ]:
from pan import ProfilingDataset, createDocProfiles, create_target_prof_trainset
from pan import preprocess

task = 'gender'
docs = createDocProfiles(dataset)
X, y = create_target_prof_trainset(docs, task)
print len(X)
#X = preprocess.preprocess(X)

In [ ]:
Profile

In [2]:
#reload(preprocess)
#reload(features)
from pan import features
from pan import preprocess
#X, y = dataset.get_data('age')
X, y = dataset.get_data('age')
print len(X)
#print X[0]
#X = preprocess.preprocess(X)
#print "$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"
#print X[0]

436


In [ ]:
import pickle
with open("../DATA/graph-twitter/graph_gender_age/X.txt", 'r') as inp:
    X = pickle.load(inp)
with open("../DATA/graph-twitter/graph_gender_age/y.txt", 'r') as yinp:
    y = pickle.load(yinp)
X = preprocess.preprocess(X)
y = [s.lower() for s in y]

In [ ]:
import json
with open("../DATA/graph-twitter/graph_gender_age/labeled.json", 'r') as inp:
    labeled = json.load(inp)
X = []
y = []
for user in labeled:
    y.append(user['gender'].lower())
    X.append(' '.join(user['texts']))
# for user in labeled:
#     y.extend([user['gender'].lower() for t in user['texts']])
#     X.extend(user['texts'])
print len(X), len(y)
#X = preprocess.preprocess(X)

## Models

In [3]:
import regex as re
import nltk
import numpy
from textblob.tokenizers import WordTokenizer
from sklearn.base import BaseEstimator, TransformerMixin
from pan.misc import _twokenize

class CharFeatureExtractor(BaseEstimator, TransformerMixin):

    """ Model that extracts Char Based Features. Counter: 32 features """

    def __init__(self, norm=True, norm_axis=0):
        self.letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
                  'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                  'u', 'v', 'w', 'x', 'y', 'z']
        self.numbers = ['0', '1', '2', '3','4', '5', '6', '7', '8', '9']
        self.special_c = ['!', '.', ':', '?', ';', ',', ')', '(', 
                          '-', '%', '$', '#', '@', '^', '&', '*', 
                          '=', '+', '/', '"', "'", '<', '>', '|',
                          '~', '`']
        self.names = ['#C', '#a-z', '#upper', '#0-9', '#w', '#\t'] + ['No_'+str(i) for i in self.special_c]
        self.norm = norm
        self.norm_axis = norm_axis

    def fit(self, X, y=None):
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count capital words in
        :returns: array

        """
        from sklearn.preprocessing import normalize
        
        final_feat = []
        for text in texts:
            #text = text.encode('utf-8', 'ignore')
            tmp = []
            tmp.append(len(text))
            if tmp[0] == 0:
                tmp = [0 for i in self.names]
            else:
                tmp.append(sum([text.lower().count(l) for l in self.letters])/float(tmp[0]))
                tmp.append(sum([l.isupper() for l in text])/float(tmp[0]))
                tmp.append(sum([text.lower().count(l) for l in self.numbers])/float(tmp[0]))
                tmp.append(text.count(' ')/float(tmp[0]))
                tmp.append(text.count('\t')/float(tmp[0]))
                tmp.extend([text.lower().count(l)/float(tmp[0]) for l in self.special_c])
            final_feat.append(tmp)
        if self.norm:
            final_feat2 = normalize(numpy.array(final_feat), norm='l1', axis=0)
            #final_feat2 = normalize(numpy.array(final_feat), norm='l1', axis=1)
            return final_feat2
            #return normalize(numpy.array(final_feat), norm='l1', axis=self.norm_axis)
        else:
            return numpy.array(final_feat) 

In [4]:
import regex as re
import nltk
import numpy
from textblob.tokenizers import WordTokenizer
from sklearn.base import BaseEstimator, TransformerMixin
from pan.misc import _twokenize

eyes = [':', ';']
noses = ['', '-', '=', 'o', 'O', '0', '>']
mouths = [')', 'D', ']', '>',  '(',  '[', '<', 'p', 'P', 'd', "|", '/', '*', '@', '!', '$', '3', 'X']
emoticons = ["*O", "*-*", "*O*", "*o*", "* *", "=}","(:;)", "{;:]", "^_^", "^-^"]
for eye in eyes:
    for nose in noses:
        for mouth in mouths:
            emoticons.append(eye+nose+mouth)


eyes = [':', ';']
noses = ['', '-']
mouths = [')', 'D', ']', '>',  '(',  '[', '<', 'p', 'P', 'd']            
tongue_no_emo = []
for nose in noses:
    for eye in eyes:
        for mouth in mouths:
            tongue_no_emo.append(eye+nose+mouth)
    
class EmoticonExtractor(BaseEstimator, TransformerMixin):

    """ Model that extracts Char Based Features. Counter: 32 features """

    def __init__(self, norm=True, norm_axis=0):
        self.emoticons = emoticons
        self.names = emoticons
        self.norm = norm
        self.norm_axis = norm_axis

    def fit(self, texts, y=None):
        """ Keep correct emoticons.

        - texts: the texts to count the emoticons 
        - returns: Nothing, just updates emoticons and names attributes

        """
        from sklearn.preprocessing import normalize
        
        final_feat = []
        for text in texts:
            #text = text.encode('utf-8', 'ignore')
            tmp = [text.count(emo) for emo in self.emoticons]
            #if ';od' in text:
            #    print 'Kai omws'
            final_feat.append(tmp)
        final_feat2 = numpy.array(final_feat)
        final_feat3 = final_feat2[:, numpy.nonzero(final_feat2.sum(axis=0))[0]]
        #print 'Indexes'
        #print numpy.nonzero(final_feat2.sum(axis=0))[0]
        #print type(numpy.nonzero(final_feat2.sum(axis=0))[0])
        #print 'Names'
        #print numpy.array(self.names).shape
        self.emoticons = numpy.array(self.emoticons)[numpy.nonzero(final_feat2.sum(axis=0))[0]]
        self.names = numpy.array(self.names)[numpy.nonzero(final_feat2.sum(axis=0))[0]]
        #print final_feat3.shape
        return self

    def transform(self, texts):
        """ transform data

        :texts: The texts to count capital words in
        :returns: array

        """
        from sklearn.preprocessing import normalize
        
        final_feat = []
        for text in texts:
            #text = text.encode('utf-8', 'ignore')
            tmp = [text.count(emo) for emo in self.emoticons]
            #if ';od' in text:
            #    print 'Kai omws'
            final_feat.append(tmp)
        final_feat2 = numpy.array(final_feat)
        #self.names = numpy.array(self.names)[numpy.nonzero(final_feat2.sum(axis=0))[0]]
        if self.norm:
            final_feat3 = normalize(numpy.array(final_feat2), norm='l1', axis=0)
            return final_feat3
            #return normalize(numpy.array(final_feat), norm='l1', axis=self.norm_axis)
        else:
            return final_feat3

In [ ]:
wanted_indexes

In [ ]:
from sklearn.cross_validation import train_test_split
split = 0.2
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=split, stratify=y, random_state=100)
print len(X_train), len(X_cv), len(X_cv) + len(X_train), len(X)

#aa = EmoticonExtractor(emoticons)
#aa = CharFeatureExtractor()
#tr = aa.fit_transform(X_train)

wanted_cols = aa.names[:5] + ['No_#', 'No_@', 'No_!', 'No_.', 'No_)', 'No_(', 'No_$', "No_`", "No_~", 'No_+']
wanted_indexes = [i for i in xrange(len(aa.names))]
# wanted_indexes = []
# for i in wanted_cols:
#     wanted_indexes.append(aa.names.index(i))

tr2 = tr[:, wanted_indexes]
cols = numpy.array(aa.names)[wanted_indexes]

import pandas as pd
data = pd.DataFrame(tr2, columns=cols.tolist())
data["class"] = y_train
print(data.describe())

In [ ]:
def make_cmap(colors, position=None, bit=False):
    '''
    make_cmap takes a list of tuples which contain RGB values. The RGB
    values may either be in 8-bit [0 to 255] (in which bit must be set to
    True when called) or arithmetic [0 to 1] (default). make_cmap returns
    a cmap with equally spaced colors.
    Arrange your tuples so that the first color is the lowest value for the
    colorbar and the last is the highest.
    position contains values from 0 to 1 to dictate the location of each color.
    '''
    import matplotlib as mpl
    import numpy as np
    bit_rgb = np.linspace(0,1,256)
    if position == None:
        position = np.linspace(0,1,len(colors))
    else:
        if len(position) != len(colors):
            sys.exit("position length must be the same as colors")
        elif position[0] != 0 or position[-1] != 1:
            sys.exit("position must start with 0 and end with 1")
    if bit:
        for i in range(len(colors)):
            colors[i] = (bit_rgb[colors[i][0]],
                         bit_rgb[colors[i][1]],
                         bit_rgb[colors[i][2]])
    cdict = {'red':[], 'green':[], 'blue':[]}
    for pos, color in zip(position, colors):
        cdict['red'].append((pos, color[0], color[0]))
        cdict['green'].append((pos, color[1], color[1]))
        cdict['blue'].append((pos, color[2], color[2]))

    cmap = mpl.colors.LinearSegmentedColormap('my_colormap',cdict,256)
    return cmap

colors = [(255,153,255), (0,0,255)] # This example uses the 8-bit RGB
my_cmap = make_cmap(colors, bit=True)

In [ ]:
grouped = data.groupby('class')
means = grouped.mean().T

import matplotlib.pyplot as plt
### BAR PLOTS OF MEAN VALUE OF FEATURES FOR EACH CLASS ######

grouped = data.groupby('class')
#plt.figure()
g2 = grouped.mean().T
g2 = g2.div(g2.sum(axis=1), axis=0)
g2.plot(kind='bar', figsize=(60,10), cmap=my_cmap, grid=True, layout='tight')
plt.savefig('test1.png', facecolor='white', dpi=100)
plt.show()

In [ ]:
list(g2.index)

In [ ]:
offline.plot.

In [ ]:
import plotly.offline as offline


offline.plot({'data': data, 
               'layout': layout, 
              },image='png')

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

colors = {'female':'rgb(255,153,255)', 'male':'rgb(0,0,255)'}

          
data = []
for col in g2.columns:
    trace1 = go.Bar(
        x=g2.index,
        y=g2[col],
        name=col,
        marker={'color':colors[col]}
        )
    data.append(trace1)
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='gender_char.png')

## Single Model

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from pan.features import SOA_Model2, SOAC_Model2
from pan import features

aa = EmoticonExtractor(emoticons)
c = CharFeatureExtractor(norm_axis=0, norm=True)
svm2 = SVC(kernel='rbf', C=1, gamma=1, class_weight='balanced', probability=False)
#w = WordFeatureExtractor(function_words_path='./English_Function_Words_Set/', norm=True, norm_axis=0)
#soacw = SOACWrapper()
#svm = LinearSVC(C=0.001,  class_weight='balanced')
#svm2 = SVC(kernel='linear', C=1, gamma=1, class_weight='balanced', probability=False)
#pipe1 = Pipeline([('EmoCount', aa), ('svm', svm2)])
#pipe2 = Pipeline([('CharCount', c), ('svm', svm2)])
#pipe2 = Pipeline([('CharCount', c), ('soac', soacw),('svm', svm2)])

num_folds = 4
split = 0.2
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=split, stratify=y, random_state=100)
print len(X_train), len(X_cv), len(X_cv) + len(X_train), len(X)
#eclf = VotingClassifier(estimators=[("0", pipe1), ('1', pipe2)], voting='soft')
#pipe2.fit(X_train, y_train)
# trained_models = []
params = {}
#params = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100]}
for model in [pipe_emo]:
    grid_search = GridSearchCV(estimator=model, param_grid=params, verbose=1, n_jobs=2, cv=num_folds, 
                                   refit=True, scoring='f1_weighted')
    grid_search.fit(X_train,y_train)
    print(grid_search.best_score_)
    print(grid_search.best_estimator_) 
best = grid_search.best_estimator_

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predict = best.predict(X_cv)
acc = accuracy_score(y_cv, predict)
conf = confusion_matrix(y_cv, predict, labels=sorted(list(set(y_cv))))
rep = classification_report(y_cv, predict, target_names=sorted(list(set(y_cv))))
print('Accuracy : {}'.format(acc))
print('Confusion matrix :\n {}'.format(conf))
print('Classification report :\n {}'.format(rep))

## Ensembles

In [5]:
class Preprocess(BaseEstimator, TransformerMixin):
    
    def __init__(self, preprocess=None, preprocess_flag=True):
        self.flag = preprocess_flag
        self.preprocess = preprocess
    def fit(self, X, y=None):
        
        return self
    
    def transform(self, X):
        
        if self.flag:
            if self.preprocess!= None:
                return self.preprocess(X)
            else:
                print "No preprocess func found"
        else:
            return X
        

### 3grams

In [6]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from pan.preprocess import preprocess

grams3 = TfidfVectorizer(analyzer='word', ngram_range=[2,2], max_features=5000) # age
svm = SVC(kernel='rbf', C=10, gamma=1, class_weight='balanced',  probability=True) # age
#grams3 = TfidfVectorizer(analyzer='word', ngram_range=[1,1], max_features=7500) # gender
#svm = LinearSVC(kernel='linear', C=1, class_weight='balanced') # gender
pipe_grams = Pipeline([('Pre', Preprocess(preprocess, True)), ('3grams',grams3), ('svm', svm)])
pipe_grams.steps

[('Pre', Preprocess(preprocess=<function preprocess at 0x7fbf6ebf90c8>,
        preprocess_flag=None)),
 ('3grams',
  TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
          dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
          lowercase=True, max_df=1.0, max_features=5000, min_df=1,
          ngram_range=[2, 2], norm=u'l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('svm', SVC(C=10, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

### SOAC

In [7]:
soac = features.SOAC_Model2(max_df=1.0, min_df=1, tokenizer_var='sklearn', max_features=None) # age
svm = SVC(kernel='rbf', C=0.1, gamma=1, class_weight='balanced',  probability=True) # age
#soac = features.SOAC_Model2(max_df=0.9, min_df=3, tokenizer_var='sklearn', max_features=None) # gender
#svm = SVC(kernel='rbf', C=0.1, gamma=1, class_weight='balanced',  probability=True) # gender

#combined = FeatureUnion([('count_tokens', countTokens), ('count_hash', countHash),
#                         ('count_urls', countUrls), ('count_replies', countReplies), 
#                          ('soa', soa), ('soac', soac)])+
#combined = FeatureUnion([('count_tokens', countTokens), ('count_hash', countHash),
#                         ('count_urls', countUrls), ('count_replies', countReplies)])
pipe_soac = Pipeline([('Pre', Preprocess(preprocess, True)), ('soac',soac), ('svm', svm)])
pipe_soac.steps

[('Pre', Preprocess(preprocess=<function preprocess at 0x7fbf6ebf90c8>,
        preprocess_flag=None)),
 ('soac', SOAC_Model2(max_df=1.0, max_features=None, min_df=1, thres=0.1,
        tokenizer_var='sklearn')),
 ('svm', SVC(C=0.1, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False))]

### LSI

In [8]:
from pan.features import LSI_Model
LSImodel = LSI_Model(num_topics=50, max_df=1.0, min_df=2, max_features=None) # age
svm = SVC(kernel='rbf', C=1, gamma=1, class_weight='balanced',  probability=True) # age
#LSImodel = LSI_Model(num_topics=50, max_df=1.0, min_df=2, max_features=None) # gender
#svm = LinearSVC(C=10, class_weight='balanced') # gender
#pipe2 = Pipeline([('counts',combined), ('svm', svm)])
pipe_lsi = Pipeline([('Pre', Preprocess(preprocess, True)), ('LSI',LSImodel), ('svm', svm)])

### LDA

In [9]:
from pan.features import LDA
LDAmodel = LDA(lib='mallet', num_topics=90)
svm = SVC(kernel='rbf', C=100, gamma=1, class_weight='balanced',  probability=True)
#pipe2 = Pipeline([('counts',combined), ('svm', svm)])
pipe_lda = Pipeline([('Pre', Preprocess(preprocess, True)), ('LDA', LDAmodel), ('svm', svm)])
pipe_lda

mallet


Pipeline(steps=[('Pre', Preprocess(preprocess=<function preprocess at 0x7fbf6ebf90c8>,
      preprocess_flag=None)), ('LDA', LDA(lib='mallet', num_topics=90)), ('svm', SVC(C=100, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

### Chars

In [10]:
c = CharFeatureExtractor(norm_axis=0, norm=True)
svm = SVC(kernel='rbf', C=1, gamma=1, class_weight='balanced', probability=True) # age
# svm = LinearSVC(C=1, class_weight='balanced') # gender
pipe_char = Pipeline([('Pre', Preprocess(preprocess, False)), ('Char', c), ('svm', svm)])

### Emo

In [11]:
emo = EmoticonExtractor()
svm = SVC(kernel='rbf', C=100, gamma=1, class_weight='balanced',  probability=True) # age
# svm = LinearSVC(C=1, class_weight='balanced') # gender
pipe_emo = Pipeline([('Pre', Preprocess(preprocess, False)), ('Emo', emo), ('svm', svm)])

In [12]:
import sys
sys.path.insert(0, "../EnsembleDiversityTests/")

from EnsembleDiversityTests import DiversityTests, BaseClassifiers
from sklearn.base import BaseEstimator, TransformerMixin

def print_overlaps(predictions, names, verbose=True):
    N = len(names)
    res = numpy.zeros([N,N])
    temp = numpy.zeros([N,N])
    for i in range(0, N):
        for j in range(i+1, N):
            temp[i,j] = len([m for l, m in enumerate(predictions[i]) if (m==predictions[j][l] and m==predictions[N-1][l])])/float(len(predictions[0]))
            res[i,j] = len([(k,v) for k,v in zip(predictions[i], predictions[j]) if k==v])/float(len(predictions[0]))
            if verbose:
                print "%s - %s : %0.3f  overlap | ground-truth coverage: %0.3f" % (names[i],  names[j], 100*res[i,j], 100*temp[i,j])
    return  [res, temp]

In [ ]:
best.steps[1][1]

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin

class SubSpaceEnsemble3(BaseEstimator, TransformerMixin):
    
    """ Utilizing the neighborhood in all representations and also ground truth model.
        Implementing a weighted voting scheme."""

    def __init__(self, models, k=3, weights= [2,1,3,0.7]):
        from sklearn.feature_extraction.text import CountVectorizer
        
        if (not models):
            raise AttributeError('Models expexts a dictonary of models \
              containg the predictions of y_true for each classifier.\ ')
        else:
            self.models = models
            # self.cv_scores = cv_scores
            self.k = k
            self.weights = weights
            self.ind2names = {}
            for i, name in enumerate(models.keys()):
                self.ind2names[i] = name
            self.counter = CountVectorizer()
            self.representations = []
            self.meta = None
            self.predictions = []
            self.true = []
            self.doc_terms = None
            self.tree = None
            self.experts = []
        

    def fit(self, X_cv, y_true=None, weights=None):
        
        from sklearn.neighbors import BallTree
        import random

        if y_true is None:
            raise ValueError('we need y labels to supervise-fit!')
        else:
            parameters = {
                    'input': 'content',
                    'encoding': 'utf-8',
                    'decode_error': 'ignore',
                    'analyzer': 'word',
                    'stop_words': 'english',
                    # 'vocabulary':list(voc),
                    #'tokenizer': tokenization,
                    #'tokenizer': _twokenize.tokenizeRawTweetText,  # self.tokenization,
                    #'tokenizer': lambda text: _twokenize.tokenizeRawTweetText(nonan.sub(po_re.sub('', text))),
                    'max_df': 1.0,
                    'min_df': 1,
                    'max_features':None
                }
            t0 = time.time()
            self.counter.set_params(**parameters)
            self.doc_terms = self.counter.fit_transform(X_cv).toarray()
            self.tree = BallTree(self.doc_terms, leaf_size=20)
            predictions = []
            for name, model in self.models.iteritems():
                predictions.append(model.predict(X_cv))
                #print len(predictions[-1])
                transf = model.steps[1][1].transform(model.steps[0][1].transform(X_cv))
                if hasattr(transf, "toarray"):
                    #print 'Exei'
                    self.representations.append(transf.toarray())
                else:
                    self.representations.append(transf)
            self.predictions = predictions
            self.true = y_true
            count = 0
            #print self.expert_scores
            #print self.experts
            print('Fit took: %0.3f seconds') % (time.time()-t0)
            return self

    def predict(self, X):
        # print "PRedict"
        # print X.shape
        X_transformed = self.counter.transform(X).toarray()
        #print type((X_transformed)[0])
        #print X_transformed.shape
        #return 0
        y_pred = []
        t0 = time.time()
        for i in range(0, X_transformed.shape[0]):
            #print X_transformed[i,:].shape
            dist, neigbors_indexes = self.tree.query(X_transformed[i,:].reshape(1,-1), self.k)  
#             print 'Sample ' + y_real[i]
            #print dist
            #print type(dist)
            #print neigbors_indexes[0]
            #print dist
            #best_model_ind = self.expert_decision(neigbors_indexes[0])
            #pass
            y_pred.append(self.expert_decision(neigbors_indexes[0],  dist, X[i]))
            
            #y_pred.append(self.models[self.ind2names[best_model_ind]].predict([X[i]])[0])
        #print y_pred
        print('Predict took: %0.3f seconds') % (time.time()-t0)
        return y_pred

    def score(self, X, y, sample_weight=None):

        from sklearn.metrics import accuracy_score
        return accuracy_score(y, self.predict(X), normalize=True)
        #return self.svc.score(self.transform_to_y(X), y, sample_weight)


    def expert_decision(self, neigbors_indexes, dist, x_sample):

        from sklearn.metrics import accuracy_score
        from collections import Counter
        from sklearn.neighbors import BallTree
        
        models_pred = []
        models_neig_pred = []
        acc = []
        t0 = time.time()
        neigbors_true = [self.true[n_i] for n_i in neigbors_indexes]
        #print('Neighbors per sample: %0.4f seconds') % (time.time()-t0)
#         print 'True'
#         print neigbors_true
        sample_predictions = []
        total_pred = []
        weights = {}
        weights['true'] = self.weights[2]
        weights['models_n'] = []
        weights['models'] = []
        for model_i in xrange(len(self.models.values())):
            ModelTree = BallTree(self.representations[model_i])
            temp_trans = self.models[self.ind2names[model_i]].steps[1][1].transform([x_sample])
            if hasattr(temp_trans, 'toarray'):
                temp_trans = temp_trans.toarray()
            _, model_neig = ModelTree.query(temp_trans, self.k)
            model_neig_pred = []
            for model_n_i in model_neig[0].tolist():
                model_neig_pred.append(self.predictions[model_i][model_n_i])
            models_neig_pred.append(model_neig_pred)
            model_pred = []
            for n_i in neigbors_indexes:
                model_pred.append(self.predictions[model_i][n_i])
            models_pred.append(model_pred)
            acc.append(accuracy_score(neigbors_true, model_neig_pred, normalize=True))
            if acc[-1] >self.weights[3]:
                # Adding neighbors predictions
                weights['models_n'].append(int(self.weights[1]/float((1-acc[-1])+0.01)))
                total_pred.extend([pred for j in xrange(weights['models_n'][-1]) for pred in model_pred])
                #print('Predicting Neighbors per sample: %0.4f seconds') % (time.time()-t0)
                # Adding sample prediction
                sample_predictions.append(self.models[self.ind2names[model_i]].predict(x_sample)[0])
                weights['models'].append(int(self.weights[0]/float((1-acc[-1])+0.01))) 
                total_pred.extend([sample_predictions[-1] for j in xrange(weights['models'][-1])])
                total_pred.extend([pred for j in xrange(weights['models'][-1]) for pred in model_neig_pred])
            #print len(x_sample)
            #print self.ind2names[model_i]
            
#                 print 'Model: ' + self.ind2names[model_i] + ' Accuracy: ' + str(accuracy_score(neigbors_true, model_neig_pred, normalize=True))
#                 print 'Predictions'
#                 print model_pred
#                 print 'Representations'
#                 print model_neig_pred
#                 print 'Sample prediction: ' + str(sample_predictions[-1])
        total_pred.extend([n for i, n in enumerate(neigbors_true) for j in xrange(int(weights['true']*(self.k-i)))])
        #print('creating votes: %0.4f seconds') % (time.time()-t0)
        data = Counter(total_pred)
        #data = Counter([k for pred in models_pred for k in pred])
#         print data
#         best_model_ind = acc.index(max(acc))
#         print 'Total pred: ' + str(data.most_common(1)[0][0])
#         print '='*50
#         #print len(total_pred)
        #return best_model_ind
        return data.most_common(1)[0][0]

In [ ]:
%load_ext autoreload
%autoreload 2

In [20]:
import sys
sys.path.insert(0, "../EnsembleDiversityTests/")
import combinations
import copy
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,precision_recall_fscore_support, classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.grid_search import GridSearchCV
from pan.features import Metaclassifier
import time
import numpy

import warnings
warnings.filterwarnings('ignore')


from pan import features
from pan import preprocess
#X, y = dataset.get_data('age')
X, y = dataset.get_data('age')


#pipe = Pipeline([('3grams',grams3), ('svm', svm)])
#pipe1 = Pipeline([('soac',soac), ('svm', svm)])
#pipe2 = Pipeline([('soa',soa), ('svm', svm)])

### AGE ###
#eclf = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ("2", pipe2)], voting='soft')
#eclfh = VotingClassifier(estimators=[("0", pipe), ('1', pipe1), ("2", pipe2)], voting='hard')
#models = [pipe,pipe1,pipe2,eclf, eclfh]
#model_names = ['3grams', 'soac', 'lda', 'voting', 'votingh']

# Base Models
base_models = [pipe_grams, pipe_soac, pipe_emo]
base_model_names = ['3grams', 'soac', 'emo']

# Meta Voting Models
eclf = VotingClassifier(estimators=zip(base_model_names, base_models), voting='soft')
eclfh = VotingClassifier(estimators=zip(base_model_names, base_models), voting='hard')
voting_dic = {'votingf':eclf, 'votingh':eclfh}
combinator_names = ['majority', 'weights', 'accuracy', 'optimal']
#meta_models_names = ['votingf', 'votingh', 'space3', 'meta'] + combinator_names
meta_models_names = ['space3', 'OLA', 'LCA', 'KNE', 'KNU'] + combinator_names
#meta_models_names = []
## all_models ##
all_models_names = base_model_names + meta_models_names


#eclf = VotingClassifier(estimators=[("0", pipe), ('1', pipe1)], voting='soft')
#eclfh = VotingClassifier(estimators=[("0", pipe), ('1', pipe1)], voting='hard')
#models = [pipe,pipe1,eclf, eclfh]
#model_names = ['3grams', 'soac', 'voting', 'votingh']

results = {'over':[]}
for name in all_models_names:
    results[name] = {'pred': [], 'conf': [], 'rep': [], 'acc': []}

num_folds = 4
train_split = 0.25
meta_split = 0.5
cv_rounds = 5
seeds = list(xrange(cv_rounds))
t0 = time.time()
t1 = t0
for j in xrange(cv_rounds):
    X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=train_split, stratify=y, random_state=seeds[j])
    for i, x in enumerate(X_train):
        if len(x)==0:
            X_train.remove(x)
            y_train.remove(y_train[i])
    for i, x in enumerate(X_cv):
        if len(x)==0:
            X_cv.remove(x)
            y_cv.remove(y_cv[i])
    if meta_split > 0:
        X_meta, X_cv, y_meta, y_cv = train_test_split(X_cv, y_cv, test_size=meta_split, stratify=y_cv)
        print len(X_train), len(X_cv), len(X_meta), len(X_cv) + len(X_train) + len(X_meta), len(X)
    else:
        print len(X_train), len(X_cv), len(X_cv) + len(X_train) , len(X)
    trained_base_models = []
    trained_all_models = []
    predictions = []
    base_predictions_cv = []
    base_predictions_meta = []
    for i, model in enumerate(base_models):
        print base_model_names[i]
        print len(X_train[0])
        model.fit(X_train,y_train)
        trained_base_models.append(model)
        trained_all_models.append(model)
        predict = model.predict(X_cv)
        predictions.append(predict)
        base_predictions_cv.append(predict)
        base_predictions_meta.append(model.predict(X_meta))
        results[base_model_names[i]]['pred'].append(predict)
        results[base_model_names[i]]['acc'].append(accuracy_score(y_cv, predict))
        results[base_model_names[i]]['conf'].append(confusion_matrix(y_cv, predict, labels=sorted(list(set(y)))))
        results[base_model_names[i]]['rep'].append(classification_report(y_cv, predict, labels=sorted(list(set(y)))))
    #trained_all_models = copy.deepcopy(trained_base_models)
    for name in meta_models_names:
        print name
        if name =='votingf' or name=='votingh':
            model = voting_dic[name]
            model.fit(X_train, y_train)
            predict = model.predict(X_cv)
        if name == 'space':
            models_for_space = {}
            cv_scores = []
            for i, base_trained_model in enumerate(trained_base_models):
                models_for_space[base_model_names[i]] = base_trained_model
                cv_scores.append(base_trained_model.score(X_meta, y_meta))
            model = combinations.SubSpaceEnsemble4_2(models_for_space, cv_scores, k=6, weights=[0.65,0.35,0.32,6], N_rand=10, rand_split=0.6)
            model.fit(X_meta, y_meta)
            predict = model.predict(X_cv)
        if name == 'space3':
            models_for_space = {}
            for i, base_trained_model in enumerate(trained_base_models):
                models_for_space[base_model_names[i]] = base_trained_model
            model = SubSpaceEnsemble3(models_for_space, k=5, weights= [2,1,3,0.6])
            model.fit(X_train, y_train)
            predict = model.predict(X_cv)
        if name == 'meta':
            model_dic = {}
            for i, base_trained_model in enumerate(trained_base_models):
                model_dic[base_model_names[i]] = base_trained_model
            model = Metaclassifier(models=model_dic, C=1.0, weights='balanced')
            model.fit(X_meta, y_meta)
            predict = model.predict(X_cv)
        if name == 'OLA' or name == 'LCA' or name== 'KNE' or name == 'KNU':
            models11 = {}
            models_tr11 = {}
            if name == 'LCA':
                for i, model1 in enumerate(trained_base_models[:]):
                    models11[base_model_names[i]] = model1
                    models_tr11[base_model_names[i]] = trained_base_models[i].steps[1][1]        
            else:   
                for i, model1 in enumerate(trained_base_models[:]):
                    models11[base_model_names[i]] = model1
                    models_tr11[base_model_names[i]] = trained_base_models[i].steps[1][1]        
            model = combinations.Neigbors_DS(models11, models_tr11, k=3, scheme=name, common_neigh=False)
            #print model
            model.fit(X_meta, y_meta)
            predict = model.predict(X_cv)
        if name in combinator_names:
            #print 'mpike'
            model = combinations.Combinator(scheme=name, weights= [1/float(len(base_predictions_meta)) for i in xrange(len(base_predictions_meta))])
            model.fit(base_predictions_meta, y_meta)
            predict = model.predict(base_predictions_cv)
        trained_all_models.append(model)
        predictions.append(predict)
        results[name]['pred'].append(predict)
        results[name]['acc'].append(accuracy_score(y_cv, predict))
        results[name]['conf'].append(confusion_matrix(y_cv, predict, labels=sorted(list(set(y)))))
        results[name]['rep'].append(classification_report(y_cv, predict, labels=sorted(list(set(y)))))
    print('Round %d took: %0.3f seconds') % (j, time.time()-t1)
    t1 = time.time()
print('Total time: %0.3f seconds') % (time.time()-t0)

326 55 55 436 436
3grams
541079
soac
541079
emo
541079
space3
Fit took: 54.221 seconds
Predict took: 71.275 seconds
OLA
Fitting time 11.10
LCA
Fitting time 8.33
KNE
Fitting time 8.28
KNU
Fitting time 8.28
majority
weights
accuracy
optimal
Round 0 took: 237.445 seconds
326 55 55 436 436
3grams
579006
soac
579006
emo
579006
space3
Fit took: 46.000 seconds
Predict took: 728.956 seconds
OLA
Fitting time 8.55
LCA
Fitting time 7.74
KNE
Fitting time 7.73
KNU
Fitting time 7.75
majority
weights
accuracy
optimal
Round 1 took: 882.217 seconds
326 55 55 436 436
3grams
453340
soac
453340
emo
453340
space3
Fit took: 47.249 seconds
Predict took: 1300.671 seconds
OLA
Fitting time 7.36
LCA
Fitting time 7.37
KNE
Fitting time 7.39
KNU
Fitting time 7.38
majority
weights
accuracy
optimal
Round 2 took: 1449.938 seconds
326 55 55 436 436
3grams
265449
soac
265449
emo
265449
space3
Fit took: 47.490 seconds
Predict took: 556.372 seconds
OLA
Fitting time 6.70
LCA
Fitting time 6.69
KNE
Fitting time 6.71
KNU
Fitt

In [16]:
for name in all_models_names:
    print '%%%%%%%%%%%%%%%%  ' + name  + '  % %%%%%%%%%%%%%%%%%%%%%%%'
    print '#################################'
    mean_acc = 0
    mean_prec = 0
    mean_rec = 0
    mean_f1 = 0
    conf = numpy.zeros([5,5])
    for i in xrange(cv_rounds):
        mean_acc += results[name]['acc'][i]
        #print results[key]['report'][i].split('     ')
        mean_prec += float(results[name]['rep'][i].split('     ')[-4][2:])
        mean_rec += float(results[name]['rep'][i].split('     ')[-3][2:])
        mean_f1 += float(results[name]['rep'][i].split('     ')[-2][2:])
        conf += results[name]['conf'][i]
    mean_acc = mean_acc/float(cv_rounds)
    mean_prec = mean_prec/float(cv_rounds)
    mean_rec = mean_rec/float(cv_rounds)
    mean_f1 = mean_f1/float(cv_rounds)
    conf = conf/float(cv_rounds)
    print('Accuracy : {}'.format(mean_acc))
    print('Precision : {}'.format(mean_prec))
    print('Recall : {}'.format(mean_rec))
    print('F1 : {}'.format(mean_f1))
    print('Confusion matrix :\n {}'.format(conf))
    print '#################################'
print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

%%%%%%%%%%%%%%%%  3grams  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.425454545455
Precision : 0.324
Recall : 0.424
F1 : 0.356
Confusion matrix :
 [[  0.2   2.4   1.4   0.    0. ]
 [  0.    6.6   9.4   1.    0. ]
 [  0.    5.4  16.6   1.    0. ]
 [  0.    2.8   7.2   0.    0. ]
 [  0.    0.    1.    0.    0. ]]
#################################
%%%%%%%%%%%%%%%%  soac  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.447272727273
Precision : 0.428
Recall : 0.446
F1 : 0.422
Confusion matrix :
 [[  0.6   2.4   0.8   0.2   0. ]
 [  1.    6.    8.2   1.8   0. ]
 [  0.8   4.   15.4   2.8   0. ]
 [  0.6   0.6   6.2   2.6   0. ]
 [  0.    0.    1.    0.    0. ]]
#################################
%%%%%%%%%%%%%%%%  emo  % %%%%%%%%%%%%%%%%%%%%%%%
#################################
Accuracy : 0.294545454545
Precision : 0.344
Recall : 0.294
F1 : 0.29
Confusion matrix :
 [[ 0.6  2.4  0.8  0.   0.2]
 [ 1.8  8.4  3.   0.4  3.4]
 [ 1.8  9.4  6.   2.  

In [ ]:
copy.deepcopy(trained_base_models[5])

In [ ]:
def